In [18]:
!git clone https://github.com/sakanaowo/PlantXMamba

Cloning into 'PlantXMamba'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 106 (delta 25), reused 96 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (106/106), 291.76 KiB | 6.79 MiB/s, done.
Resolving deltas: 100% (25/25), done.


In [19]:
!git clone https://github.com/sakanaowo/PlantXViT

Cloning into 'PlantXViT'...
remote: Enumerating objects: 104825, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 104825 (delta 6), reused 20 (delta 4), pack-reused 104802 (from 1)
Receiving objects: 100% (104825/104825), 2.45 GiB | 43.00 MiB/s, done.
Resolving deltas: 100% (30447/30447), done.
Updating files: 100% (104353/104353), done.


In [20]:
from PlantXViT.utils.config_loader import load_config
config=load_config('PlantXViT/configs/config.yaml')

In [21]:
import pandas as pd
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset
from torchvision import transforms
import pickle

Apple train attempt

In [22]:
apple_config = config['dataset']['apple']
img_dir = apple_config['data_dir']
csv_path = apple_config['csv_path']
label_encoder_path = apple_config['label_encoder']
image_size = tuple(apple_config['image_size'])

In [23]:
csv_path='./PlantXViT/data/raw/plant-pathology-2020-fgvc7/train.csv'
label_encoder_path='./PlantXViT/data/processed/apple_label_encoder.pkl'
print(label_encoder_path)

./PlantXViT/data/processed/apple_label_encoder.pkl


In [24]:
df = pd.read_csv(csv_path)
df['label'] = df[['healthy', 'multiple_diseases', 'rust', 'scab']].idxmax(axis=1)

In [25]:
# encode label
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

In [26]:
# split train/val
train_df, val_df = train_test_split(df, test_size=0.2,
                                    stratify=df['label'],
                                    random_state=42)

# Encode labels
label_encoder = LabelEncoder()
train_df['label_idx'] = label_encoder.fit_transform(train_df['label'])
val_df['label_idx'] = label_encoder.transform(val_df['label'])

In [27]:
image_dir = "./PlantXViT/data/raw/plant-pathology-2020-fgvc7/images"
train_df = pd.read_csv("./PlantXViT/data/processed/apple/apple_train.csv")
val_df = pd.read_csv("./PlantXViT/data/processed/apple/apple_val.csv")

In [28]:
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

class AppleDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform):
        self.df = dataframe
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.image_dir, row['image_id'] + ".jpg")
        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)
        label = torch.tensor(row['label_idx'])
        return image, label

In [29]:
from torch.utils.data import DataLoader

batch_size = config["training"]["batch_size"]

train_dataset = AppleDataset(train_df, img_dir, transform)
val_dataset = AppleDataset(val_df, img_dir, transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [30]:
%cd PlantXMamba

/kaggle/working/PlantXMamba/PlantXMamba


install mamba

In [84]:
!pip install -q -r requirements.txt

In [85]:
!pip show causal-conv1d 

Name: causal-conv1d
Version: 1.5.0.post8
Summary: Causal depthwise conv1d in CUDA, with a PyTorch interface
Home-page: https://github.com/Dao-AILab/causal-conv1d
Author: Tri Dao
Author-email: tri@tridao.me
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: ninja, packaging, torch
Required-by: 


In [32]:
!pip install mamba-ssm --no-build-isolation

In [62]:
!git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 560 bytes | 560.00 KiB/s, done.
From https://github.com/sakanaowo/PlantXMamba
   ec35d99..8e76479  main       -> origin/main
Updating ec35d99..8e76479
Fast-forward
 mamba_ssm/__init__.py | 12 +++++++++++-
 1 file changed, 11 insertions(+), 1 deletion(-)


In [77]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision.models import VGG16_Weights
from mamba_ssm import Mamba
import os
import sys


class InceptionBlock(nn.Module):
    def __init__(self, in_channels=128):
        super(InceptionBlock, self).__init__()
        self.branch1x1 = nn.Sequential(
            nn.Conv2d(in_channels, 128, kernel_size=1),
            nn.ReLU(),
            nn.BatchNorm2d(128)
        )
        self.branch3x3 = nn.Sequential(
            nn.Conv2d(in_channels, 96, kernel_size=1),
            nn.ReLU(),
            nn.BatchNorm2d(96),
            nn.Conv2d(96, 128, kernel_size=(3, 1), padding=(1, 0)),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, kernel_size=(1, 3), padding=(0, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(128)
        )
        self.branch5x5 = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 96, kernel_size=(3, 1), padding=(1, 0)),
            nn.ReLU(),
            nn.BatchNorm2d(96),
            nn.Conv2d(96, 96, kernel_size=(1, 3), padding=(0, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(96),
            nn.Conv2d(96, 192, kernel_size=(3, 1), padding=(1, 0)),
            nn.ReLU(),
            nn.BatchNorm2d(192),
            nn.Conv2d(192, 192, kernel_size=(1, 3), padding=(0, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(192)
        )
        self.branch_pool = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, 64, kernel_size=1),
            nn.ReLU(),
            nn.BatchNorm2d(64)
        )

    def forward(self, x):
        b1 = self.branch1x1(x)
        b2 = self.branch3x3(x)
        b3 = self.branch5x5(x)
        b4 = self.branch_pool(x)
        return torch.cat([b1, b2, b3, b4], dim=1)

class PatchEmbedding(nn.Module):
    def __init__(self, in_channels, patch_size=5, emb_size=16):
        super().__init__()
        self.patch_size = patch_size
        self.emb_size = emb_size
        self.proj = nn.Linear(in_channels * patch_size * patch_size, emb_size)

    def forward(self, x):
        B, C, H, W = x.shape
        x = x.unfold(2, self.patch_size, self.patch_size).unfold(3, self.patch_size, self.patch_size)
        x = x.permute(0, 2, 3, 1, 4, 5).contiguous()
        x = x.view(B, -1, C * self.patch_size * self.patch_size)
        return self.proj(x)

class MambaBlock(nn.Module):
    def __init__(self, emb_size=16, d_state=64, d_conv=4, expand=4):
        super().__init__()
        self.norm = nn.LayerNorm(emb_size)
        self.mamba = Mamba(
            d_model=emb_size,
            d_state=d_state,
            d_conv=d_conv,
            expand=expand
        )
        print("Mamba instance:", self.mamba)
        self.mlp = nn.Sequential(
            nn.Linear(emb_size, emb_size * 2),
            nn.GELU(),
            nn.Linear(emb_size * 2, emb_size)
        )
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        residual = x
        x = self.norm(x)
        print("Input to Mamba:", x.shape)
        print("Mamba callable:", callable(self.mamba))
        x = self.mamba(x)
        x = residual + x
        x = x + self.mlp(self.norm(x))
        return self.dropout(x)

class PlantXMamba(nn.Module):
    def __init__(self, num_classes=4, patch_size=5, emb_size=16, num_blocks=4, dropout=0.1):
        super().__init__()
        vgg = models.vgg16(weights=VGG16_Weights.DEFAULT)
        self.vgg_block = nn.Sequential(*list(vgg.features[:10]))
        self.inception = InceptionBlock(in_channels=128)
        self.patch_embed = PatchEmbedding(in_channels=512, patch_size=patch_size, emb_size=emb_size)
        self.transformer = nn.Sequential(*[MambaBlock(emb_size=emb_size, d_state=64, d_conv=4, expand=4) for _ in range(num_blocks)])
        self.norm = nn.LayerNorm(emb_size)
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.classifier = nn.Linear(emb_size, num_classes)

    def forward(self, x):
        x = self.vgg_block(x)
        x = self.inception(x)
        x = self.patch_embed(x)
        x = self.transformer(x)
        x = self.norm(x)
        x = x.permute(0, 2, 1)
        x = self.global_pool(x).squeeze(-1)
        return self.classifier(x)

In [78]:
def test_plantxmamba(batch_size, device, input_size=(3, 224, 224), num_classes=4, patch_size=5, emb_size=16):
    print(f"Testing PlantXMamba with batch_size={batch_size} on device={device}")
    
    model = PlantXMamba(
        num_classes=num_classes,
        patch_size=patch_size,
        emb_size=emb_size,
        num_blocks=4,
        dropout=0.1
    ).to(device)
    
    model_device = next(model.parameters()).device
    print(f"Model is on device: {model_device}")
    
    x = torch.randn(batch_size, *input_size).to(device)
    print(f"Input shape: {x.shape}, Input device: {x.device}")
    
    output = None
    try:
        model.eval()
        with torch.no_grad():
            output = model(x)
        print(f"Output shape: {output.shape}")
        print(f"Output device: {output.device}")
        
        expected_output_shape = (batch_size, num_classes)
        assert output.shape == expected_output_shape, \
            f"Expected output shape {expected_output_shape}, but got {output.shape}"
        print("Test passed successfully!")
        
    except Exception as e:
        print(f"Error during testing: {str(e)}")
    
    return output

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    print("\n--- Test Case 1: Batch size = 1 ---")
    output1 = test_plantxmamba(batch_size=1, device=device, patch_size=5, emb_size=16)
    
    print("\n--- Test Case 2: Batch size = 4 ---")
    output2 = test_plantxmamba(batch_size=4, device=device, patch_size=5, emb_size=16)

Using device: cuda

--- Test Case 1: Batch size = 1 ---
Testing PlantXMamba with batch_size=1 on device=cuda
Mamba instance: Mamba(
  (in_proj): Linear(in_features=16, out_features=128, bias=False)
  (conv1d): Conv1d(64, 64, kernel_size=(4,), stride=(1,), padding=(3,), groups=64)
  (act): SiLU()
  (x_proj): Linear(in_features=64, out_features=129, bias=False)
  (dt_proj): Linear(in_features=1, out_features=64, bias=True)
  (out_proj): Linear(in_features=64, out_features=16, bias=False)
)
Mamba instance: Mamba(
  (in_proj): Linear(in_features=16, out_features=128, bias=False)
  (conv1d): Conv1d(64, 64, kernel_size=(4,), stride=(1,), padding=(3,), groups=64)
  (act): SiLU()
  (x_proj): Linear(in_features=64, out_features=129, bias=False)
  (dt_proj): Linear(in_features=1, out_features=64, bias=True)
  (out_proj): Linear(in_features=64, out_features=16, bias=False)
)
Mamba instance: Mamba(
  (in_proj): Linear(in_features=16, out_features=128, bias=False)
  (conv1d): Conv1d(64, 64, kernel_